<a href="https://colab.research.google.com/github/Igirisu861/Parcial1_Ciberseguridad_DianaRuiz/blob/main/Parcial1_Ejercicio2_DianaRuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejercicio 2**

---

Mediante el algoritmo RSA Alice firmará digitalmente el contrato NDA.pdf que se
adjunta en la actividad, deberá cumplir con las siguientes instrucciones:

* Alice firmará digitalmente el contrato h(M) usando el algoritmo RSA
mediante su llave privada.
* Una vez obtenida la firma Alice agregará dicha firma (cadena de caracteres)
al archivo PDF y se lo enviará a la Autoridad Certificadora (AC).
* La AC obtendrá el HASH del documento original y verificará la firma usando
la llave pública de Alice.
* La AC firmará el documento con su llave privada y se la agrega al PDF y se la
envía a Bob.
* Bob obtiene el HASH del documento PDF y verifica la firma de la AC con la
llave pública de AC.

## **Importamos librerías**

Adicionalmente se importó fitz, librería para modificar el PDF con la firma digital.

In [36]:
!pip install pycryptodome
!pip install PyMuPDF

In [37]:
import Crypto.Util.number
import random
import hashlib
import fitz

## **Métodos a utilizar**

### **Método para generar llaves**

In [38]:
def generateKeys():
  e = 65537
  p = Crypto.Util.number.getPrime(1024)
  q = Crypto.Util.number.getPrime(1024)
  pub = p*q
  phi = (p-1)*(q-1)
  priv = pow(e, -1, phi)
  return (pub, e), (pub, priv)

### **Método para generar la firma**

In [39]:
def signHash(privKey, docPath):
  pub, priv = privKey
  with open(docPath, 'rb') as f:
    data = f.read()
  hashVal = int.from_bytes(hashlib.sha256(data).digest(), byteorder='big')
  signature = pow(hashVal, priv, pub)
  return signature

### **Método para añadirle la firma al documento pdf**

In [40]:
def addSignToDoc(pdfPath, signature, pdfResult):
    doc = fitz.open(pdfPath)
    page = doc[-1]

    x_start = 50
    y_start = 500
    line_spacing = 20

    text = page.get_text("text")
    existing_signatures = text.count("Firma RSA:")
    y_offset = y_start + (existing_signatures * line_spacing)
    signature_str = f"Firma RSA: {signature}"

    page.insert_text((x_start, y_offset), signature_str, fontsize=10)

    y_offset += line_spacing

    doc.save(pdfResult)

### **Método para verificar la firma digital**

In [41]:
def verifySignature(pubKey, docPath, signature):
  pub,e = pubKey
  with open(docPath, 'rb') as f:
    data = f.read()
  hashVal = int.from_bytes(hashlib.sha256(data).digest(), byteorder='big')
  decryptedSignature = pow(signature, e, pub)
  return hashVal == decryptedSignature

## **Proceso de intercambio del documento firmado**
---
Primero Alice genera sus llaves y firma el PDF del NDA original. Además, le agrega su firma al documento.


In [42]:
pubA, privA = generateKeys()
docPath = '/content/NDA.pdf'
aliceSign = signHash(privA, docPath)
addSignToDoc(docPath, aliceSign, 'NDASignedByAlice.pdf')

Ahora, creamos las llaves de la AC y esta verifica el documento firmado por Alice. En caso de ser verificado, la AC vuelve a firmar ese documento y añade su firma al PDF

In [43]:
acPub, acPriv = generateKeys()

if verifySignature(pubA, docPath, aliceSign):
  print('Firma verificada por la AC')
  addSignToDoc('NDASignedByAlice.pdf', acSign, 'NDASignedByAC.pdf')
  acSign = signHash(acPriv, 'NDASignedByAC.pdf')
else:
  print('Firma no válida')

Firma verificada por la AC


Finalmente, Bob recibe el documento firmado por la AC y lo verifica sin necesidad de usar la llave de Alice, sólo usa la llave pública de la AC

In [44]:
#Bob recibe el documento desde la AC con su respectiva firma
# y verifica
if verifySignature(acPub, 'NDASignedByAC.pdf', acSign):
  print('Firma verificada por Bob')
else:
  print('Firma no válida')

Firma verificada por Bob


## **Código completo**

In [45]:
import Crypto.Util.number
import random
import hashlib
import fitz

#métodos a utilizar

def generateKeys():
  e = 65537
  p = Crypto.Util.number.getPrime(1024)
  q = Crypto.Util.number.getPrime(1024)
  pub = p*q
  phi = (p-1)*(q-1)
  priv = pow(e, -1, phi)
  return (pub, e), (pub, priv)

def signHash(privKey, docPath):
  pub, priv = privKey
  with open(docPath, 'rb') as f:
    data = f.read()
  hashVal = int.from_bytes(hashlib.sha256(data).digest(), byteorder='big')
  signature = pow(hashVal, priv, pub)
  return signature

def addSignToDoc(pdfPath, signature, pdfResult):
    doc = fitz.open(pdfPath)
    page = doc[-1]

    # Definir posición base y márgenes
    x_start = 50  # Margen izquierdo
    y_start = 500  # Posición inicial en Y
    line_spacing = 20  # Espaciado entre líneas

    # Contar firmas existentes para posicionar la nueva
    text = page.get_text("text")
    existing_signatures = text.count("Firma RSA:")
    y_offset = y_start + (existing_signatures * line_spacing)
    signature_str = f"Firma RSA: {signature}"

    # Insertar la firma en el documento considerando las posiciones y modifica
    #el tamaño de la fuente de hacer falta
    page.insert_text((x_start, y_offset), signature_str, fontsize=10)

    # Ajustar la posición para la siguiente firma si es necesario
    y_offset += line_spacing

    # Guardar el documento con la nueva firma
    doc.save(pdfResult)


def verifySignature(pubKey, docPath, signature):
  pub,e = pubKey
  with open(docPath, 'rb') as f:
    data = f.read()
  hashVal = int.from_bytes(hashlib.sha256(data).digest(), byteorder='big')
  decryptedSignature = pow(signature, e, pub)
  return hashVal == decryptedSignature


#Comienza proceso
#Generamos las llaves de Alice
pubA, privA = generateKeys()

docPath = '/content/NDA.pdf'

#Se crea el hash del documento y se calcula la firma de Alice
#usando sus llaves
aliceSign = signHash(privA, docPath)

#se agrega al documento pdf la firma
addSignToDoc(docPath, aliceSign, 'NDASignedByAlice.pdf')

#generamos las llaves de la AC
acPub, acPriv = generateKeys()

if verifySignature(pubA, docPath, aliceSign):
  print('Firma verificada por la AC')
  addSignToDoc('NDASignedByAlice.pdf', acSign, 'NDASignedByAC.pdf')
  acSign = signHash(acPriv, 'NDASignedByAC.pdf')
else:
  print('Firma no válida')

#Bob recibe el documento desde la AC con su respectiva firma
# y verifica
if verifySignature(acPub, 'NDASignedByAC.pdf', acSign):
  print('Firma verificada por Bob')
else:
  print('Firma no válida')

Firma verificada por la AC
Firma verificada por Bob
